In [ ]:
# Celda 1: Correlación de Pearson

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# --- Configuración ---
# Se define la ruta del archivo.
# Se utiliza r'...' (raw string) para interpretar correctamente las barras invertidas '\' en Windows.
ruta_archivo = r'data\water_quality.csv'

# --- Carga y Preparación de Datos ---
try:
    # Se leen los datos del archivo CSV
    df = pd.read_csv(ruta_archivo)
    
    # Se convierte la columna 'Timestamp' a datetime (opcional, pero buena práctica)
    # df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    
    # Se seleccionan solo las columnas numéricas para la correlación.
    # Se excluye 'Timestamp' ya que no es una variable numérica para este análisis.
    df_numeric = df.drop(columns=['Timestamp'])

    # --- Cálculo de Correlación ---
    # Se calcula la matriz de correlación de Pearson
    corr_matrix = df_numeric.corr(method='pearson')

    # --- Visualización ---
    # Se configura el tamaño de la figura
    plt.figure(figsize=(10, 8))
    
    # Se genera el heatmap usando seaborn
    sns.heatmap(
        corr_matrix, 
        annot=True,     # Se muestran los valores numéricos en cada celda
        cmap='coolwarm',# Se define el mapa de color (rojo=positivo, azul=negativo)
        fmt='.2f',      # Se formatean los números a 2 decimales
        linewidths=.5   # Se añade una leve separación entre celdas
    )
    
    # Se añade un título al gráfico
    plt.title('Heatmap de Correlación de Pearson')
    
    # Se muestra el gráfico
    plt.show()

except FileNotFoundError:
    print(f"Error: No se encontró el archivo en la ruta especificada: {ruta_archivo}")
except Exception as e:
    print(f"Ocurrió un error al procesar la Celda 1: {e}")

In [ ]:
# Celda 2: Información Mutua (MI)

# Se importan las librerías necesarias
import numpy as np
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Se comprueba si la variable 'df_numeric' existe (creada en la Celda 1)
if 'df_numeric' in locals():
    
    # Se obtienen los nombres de las columnas
    columnas = df_numeric.columns
    n_cols = len(columnas)
    
    # Se inicializa una matriz vacía (DataFrame) para almacenar los resultados de MI
    mi_matrix = pd.DataFrame(np.zeros((n_cols, n_cols)), columns=columnas, index=columnas)

    # --- Cálculo de Información Mutua ---
    # Se debe iterar sobre cada par de columnas (i, j)
    # MI(A,B) requiere X (features, 2D) e y (target, 1D)
    
    print("Calculando Información Mutua (esto puede tardar unos segundos)...")
    
    for i in columnas:
        for j in columnas:
            
            # Se prepara X (columna i) e y (columna j)
            X_mi = df_numeric[[i]] # X debe ser 2D (DataFrame)
            y_mi = df_numeric[j]   # y debe ser 1D (Series)
            
            # Se calcula la MI usando 'mutual_info_regression' para variables continuas
            # El resultado es un array, se extrae el primer (y único) valor
            mi_val = mutual_info_regression(X_mi, y_mi, random_state=42)
            
            # Se almacena el valor en la matriz
            mi_matrix.loc[i, j] = mi_val[0]

    print("Cálculo de MI completado.")

    # --- Visualización ---
    # Se configura el tamaño de la figura
    plt.figure(figsize=(10, 8))
    
    # Se genera el heatmap para la Información Mutua
    sns.heatmap(
        mi_matrix, 
        annot=True,     # Se muestran los valores
        cmap='viridis', # Se usa un mapa de color secuencial (0 a max)
        fmt='.3f',      # Se usan 3 decimales para MI
        linewidths=.5
    )
    
    # Se añade un título
    plt.title('Heatmap de Información Mutua (variables continuas)')
    
    # Se muestra el gráfico
    plt.show()

else:
    print("Error: El DataFrame 'df_numeric' no fue definido.")
    print("Asegúrese de ejecutar la Celda 1 exitosamente primero.")